In [1]:
from keras.models import Sequential
from keras.layers import Dense,LSTM,GRU,Dropout,BatchNormalization
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.layers import Embedding
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import os
from keras.initializers import Constant
import pandas as pd
from keras.layers import LeakyReLU
from keras.layers import Bidirectional
from keras.optimizers import adam
from keras.callbacks import ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
import sklearn
from keras.layers import Activation
from sklearn import model_selection
from keras import utils

Using TensorFlow backend.


## Read the dataset

In [0]:
df0 = pd.read_csv('train.csv')
df1= pd.read_csv('test.csv')

## Output labels

In [0]:
y = df0['label'].to_numpy()

## Tokenize

In [4]:
maxlen= 70

ser0=df0.text
ser1=df1.text
ser3 = pd.concat([ser0, ser1], ignore_index=True)
tk = Tokenizer(lower = True)
tk.fit_on_texts(ser3)
x_seq0 = tk.texts_to_sequences(ser0)
x_seq1 = tk.texts_to_sequences(ser1)
x_pad0 = pad_sequences(x_seq0,maxlen=maxlen)
x_pad1 = pad_sequences(x_seq1,maxlen=maxlen)

# Find the maximum length of any review
print('Maximum review length: {}'.format(len(max(x_seq0, key=len))))

vocab_size = len(tk.word_counts.keys())+1

# Counting the number of unique words in all reviews
word_index = tk.word_index
print('Found %s unique tokens.' % len(word_index))

Maximum review length: 44
Found 37645 unique tokens.


## GloVe

In [5]:
!wget http://nlp.stanford.edu/data/glove.42B.300d.zip
!unzip glove*.zip

--2019-10-23 06:54:57--  http://nlp.stanford.edu/data/glove.42B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.42B.300d.zip [following]
--2019-10-23 06:55:03--  https://nlp.stanford.edu/data/glove.42B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip [following]
--2019-10-23 06:55:03--  http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1877800501 (1.7G) [application/zip]
Sav

In [6]:
# processing the glove embedding txt file
GLOVE_DIR = ''
embeddings_index = {}
with open(os.path.join(GLOVE_DIR, 'glove.42B.300d.txt')) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, 'f', sep=' ')
        embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))

Found 1917494 word vectors.


In [0]:
MAX_NUM_WORDS = 25000
embedding_size = 300

In [0]:
# preparing embedding matrix
num_words = min(MAX_NUM_WORDS, len(word_index)) + 1
embedding_matrix = np.zeros((num_words, embedding_size))
for word, i in word_index.items():
    if i > MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

## Train Test Split

In [0]:
# splitting training and test data
x_train,x_test,y_train,y_test = train_test_split(x_pad0 , y , test_size=0.2)

In [0]:
# splitting training and validation data
valid_size = 2048
x_train1 = x_train[valid_size:]
y_train1 = y_train[valid_size:]
x_val = x_train[:valid_size]
y_val = y_train[:valid_size]

## Beginning the model

In [0]:
filepath="weights.{epoch:02d}-{val_loss:.2f}.h5,"
checkpoint = ModelCheckpoint(filepath, verbose=1, save_best_only=False,save_weights_only=True)
callbacks_list = [checkpoint]

## Model

In [0]:
# create a RNN model
def build_model(units1, units2 , units3, drop1=0.4,  drop2=0.4):
  model = Sequential()
  
  # Embedding Layer
  model.add(Embedding(vocab_size,embedding_size,embeddings_initializer=Constant(embedding_matrix),input_length = maxlen,trainable=True))
  
  # Layer 1
  model.add(Bidirectional(LSTM(units1,return_sequences=False)))
  model.add(Dropout(drop1))

  # Layer 2
  model.add(Dense(units2));
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Dropout(drop2))

  # Layer 3
  model.add(Dense(units3))
  model.add(BatchNormalization())
  model.add(LeakyReLU());
  model.add(Dropout(drop2))

  # Layer 4
  model.add(Dense(1,activation = 'sigmoid'))
  model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
  return model

## Train

In [13]:
model = build_model(200,70,20)





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [14]:
# begin training on dataset
batch_size = 512
num_epochs = 6
model.fit(x_train1,y_train1,validation_data= (x_val,y_val),batch_size = batch_size, epochs= num_epochs,callbacks=callbacks_list)




Train on 29776 samples, validate on 2048 samples
Epoch 1/6





29776/29776 [==============================] - 34s 1ms/step - loss: 0.6880 - acc: 0.6104 - val_loss: 0.5601 - val_acc: 0.7144

Epoch 00001: saving model to weights.01-0.56.h5,
Epoch 2/6
29776/29776 [==============================] - 26s 880us/step - loss: 0.5261 - acc: 0.7409 - val_loss: 0.5613 - val_acc: 0.7305

Epoch 00002: saving model to weights.02-0.56.h5,
Epoch 3/6
29776/29776 [==============================] - 26s 880us/step - loss: 0.4060 - acc: 0.8232 - val_loss: 0.4965 - val_acc: 0.7671

Epoch 00003: saving model to weights.03-0.50.h5,
Epoch 4/6
29776/29776 [==============================] - 26s 882us/step - loss: 0.3134 - acc: 0.8726 - val_loss: 0.4860 - val_acc: 0.7964

Epoch 00004: saving model to weights.04-0.49.h5,
Epoch 5/6
29776/29776 [==============================] - 26s 881us/step - loss: 0.2411 - acc: 0.9088 - val_loss: 0.5477 - val_acc: 0.7695

Epoch 00005: saving model to weights.05-0.55.h5,
Epoch

In [0]:
model.save_weights('weights.h5')

In [16]:
# check accuracy on test data
scores = model.evaluate(x_test,y_test,verbose=1)
print(scores)
print("accuracy:",scores[1])

7956/7956 [==============================] - 29s 4ms/step
[0.5703581128065283, 0.7926093514628479]
accuracy: 0.7926093514628479


## Submission prediction

In [0]:
length = 1975
l=[i for i in range(length)]

y_sub = model.predict(x_pad1)

In [0]:
y_sub1 = y_sub >= 0.575
y_sub2 = y_sub1.reshape((1975,))
y_sub3=y_sub2.astype('int64')

submission=pd.DataFrame({'Id':l,'label':y_sub3})
submission.to_csv("submission.csv",index=False)

In [0]:
# Compare with prev best submission
sub0 = pd.read_csv('submission(23).csv')
sub3 = pd.read_csv('submission.csv')
cmp = sub0['label'] != sub3['label']
cmp = cmp.to_numpy()
print(np.sum(cmp,axis=0))